## This notebook contains:
* Running Llama2 in the cloud hosted on Replicate
* Using LangChain to ask Llama general questions and follow up questions
* Using LangChain to load PDF docs - (Kenya Law Documents) - and chat about it.
* The end result will be a chatbot that will be able answer questions about the data not publicly available when Llama2 was trained, or about your own data. RAG is one way to prevent LLM's hallucination

Let's start by installing the necessary packages:
- sentence-transformers for text embeddings
- chromadb gives us database capabilities 
- langchain provides necessary RAG tools for this demo

And setting up the Replicate token.

In [ ]:
%pip install langchain replicate sentence-transformers chromadb pypdf

Next we call the Llama 2 model from replicate. In this example we will use the llama 2 13b chat model. You can find more Llama 2 models by searching for them on the [Replicate model explore page](https://replicate.com/explore?query=llama).

The model is added in the format: model_name/version


In [31]:
# set up the Replicate API token
from getpass import getpass
import os

print("Getting Replicate API token...")
REPLICATE_API_TOKEN = "r8_aARpJdbqdFixYHVQ9LvCvaoRA0Svg8j2Pw5W3" 
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN
print("Replicate API token set.")

# initialize the Llama2 model

from langchain.llms import Replicate

print("Initializing Llama2 model...")
llama2_13b = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"
llm = Replicate(
    model=llama2_13b,
    model_kwargs={"temperature": 0.01, "top_p": 1, "max_new_tokens":500}
)
print("Llama2 model initialized.")


Getting Replicate API token...
Replicate API token set.
Initializing Llama2 model...
Llama2 model initialized.


With the model set up, it is now possible to ask some questions.

In [4]:
question = "who wrote the book Innovator's dilemma?"
answer = llm.invoke(question)
print(answer)

 Hello! I'd be happy to help you with your question. The book "The Innovator's Dilemma" was written by Clayton Christensen, an American author and academic who is known for his work on innovation and disruptive technologies. He is a professor at Harvard Business School and has written several other influential books on business and leadership. Is there anything else you would like to know about this topic?


We will then try to follow up the response with a question asking for more information on the book. 

Since the chat history is not passed on Llama doesn't have the context and doesn't know this is more about the book thus it treats this as new query.


In [5]:
# chat history not passed so Llama doesn't have the context and doesn't know this is more about the book
followup = "tell me more"
followup_answer = llm(followup)
print(followup_answer)

 Hello! As a helpful assistant, I'm here to assist you with any questions or tasks you may have. Whether it's providing information on a wide range of topics, helping you with a project or task, or simply being a listening ear, I'm here to help in any way I can.

I have access to a vast amount of knowledge and resources, so if there's something you're curious about or need help with, feel free to ask! Some examples of things I can help with include:

* Answering questions on a variety of topics such as history, science, technology, health, and more
* Providing guidance on how to complete a task or project
* Offering suggestions and ideas for creative projects or brainstorming sessions
* Assisting with language translation and communication
* And much more!

Is there anything specific you would like to know or discuss? Please don't hesitate to ask, and I'll do my best to assist you.


[`ConversationBufferMemory`](https://python.langchain.com/docs/modules/memory/types/buffer) is used to pass the chat history to the model and give it the capability to handle follow up questions.

In [32]:
# using ConversationBufferMemory to pass memory (chat history) for follow up questions
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

Once this is set up, let us repeat the steps from before and ask the model a simple question.

Then we pass the question and answer back into the model for context along with the follow up question.

In [7]:
# restart from the original question
answer = conversation.predict(input=question)
print(answer)

 Ah, you're asking about "The Innovator's Dilemma," the iconic book written by Clayton Christensen! He is a renowned author, professor, and business consultant known for his groundbreaking work in the fields of innovation and disruptive technologies.

Published in 1997, "The Innovator's Dilemma" explores why successful companies often struggle to adapt to new technologies and business models that ultimately disrupt their industries. The book introduces the concept of "disruptive innovation," which describes how small, unassuming ideas can eventually upend entire markets and leave established players behind.

Clayton Christensen has since become one of the most influential voices in the business world, and his ideas have been applied across various sectors, from technology and healthcare to finance and education. His follow-up books, such as "The Innovator's Solution" and "Competing Against Luck," further expand on these concepts and offer practical guidance for leaders looking to stay 

In [ ]:
# pass context (previous question and answer) along with the follow up "tell me more" to Llama who now knows more of what
memory.save_context({"input": question},
                    {"output": answer})
followup_answer = conversation.predict(input=followup)
print(followup_answer)

Load the documents from the data path

Using Llama 2 to answer questions using documents for context. 
This gives us the ability to update Llama 2's knowledge thus giving it better context without needing to fine-tune. 

In [34]:
from langchain.document_loaders import PyPDFDirectoryLoader

DATA_PATH = 'data'

In [35]:
loader = PyPDFDirectoryLoader(DATA_PATH)
docs = loader.load()

In [36]:
import os

# Check the number of PDF files in the data directory
pdf_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.pdf')]
print(f"Number of PDF files: {len(pdf_files)}")


Number of PDF files: 3


Storing the documents. There are more than 30 vector stores (DBs) supported by LangChain. 
In this case [Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma) is used which is light-weight and in memory so it's easy to get started with.

We will also import the HuggingFaceEmbeddings and RecursiveCharacterTextSplitter to assist in storing the documents.

In [37]:
from langchain.vectorstores import Chroma

# embeddings are numerical representations of the question and answer text
from langchain.embeddings import HuggingFaceEmbeddings

# use a common text splitter to split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

To store the documents, we will need to split them into chunks using [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter) and create vector representations of these chunks using [`HuggingFaceEmbeddings`](https://www.google.com/search?q=langchain+hugging+face+embeddings&sca_esv=572890011&ei=ARUoZaH4LuumptQP48ah2Ac&oq=langchian+hugg&gs_lp=Egxnd3Mtd2l6LXNlcnAiDmxhbmdjaGlhbiBodWdnKgIIADIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCkjeHlC5Cli5D3ABeAGQAQCYAV6gAb4CqgEBNLgBAcgBAPgBAcICChAAGEcY1gQYsAPiAwQYACBBiAYBkAYI&sclient=gws-wiz-serp) on them before storing them into our vector database. 

In general, you should use larger chuck sizes for highly structured text such as code and smaller size for less structured text. You may need to experiment with different chunk sizes and overlap values to find out the best numbers.

In [38]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(docs)

# create the vector db to store all the split chunks as embeddings
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
)

We then use ` RetrievalQA` to retrieve the documents from the vector database and give the model more context on Llama 2, thereby increasing its knowledge.

For each question, LangChain performs a semantic similarity search of it in the vector db, then passes the search results as the context to Llama to answer the question.

In [41]:
# use LangChain's RetrievalQA, to associate Llama with the loaded documents stored in the vector db
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)


question = "Tell me what you know about the Obi Van Passi Government Marriage Act of 2025"
result = qa_chain({"query": question})
print(result['result'])

 Based on the context provided, I can provide information about the Obi Van Passi Government Marriage Act of 2025. This act was enacted to regulate marriage and procreation in order to promote population growth and stability within the nation.

Here are some key points from the act:

1. Minimum number of children: Each adult male citizen is required to father a minimum of ten children during their lifetime. Failure to meet this requirement will result in penalties.
2. Open marriages: Open marriages, where spouses are permitted to engage in romantic or sexual relationships with individuals outside of their marriage, are legal and permissible for adult males. Female spouses are not permitted to engage in open marriages.
3. Procreation obligations: The government shall monitor the procreation obligations outlined in Section 3 of the act, and citizens shall be required to report the birth of each child to the government within thirty days of the child's birth. Failure to report the birth o

Now, lets bring it all together by incorporating follow up questions.

First we ask a follow up questions without giving the model context of the previous conversation. 
Without this context, the answer we get does not relate to our original question.

In [40]:
# no context passed so Llama2 doesn't have enough context to answer so it lets its imagination go wild
result = qa_chain({"query": "what are its use cases?"})
print(result['result'])

 Based on the provided context, it appears that the term "2023 Finance !91 No. 4 of a public character" refers to a specific type of entity or organization that is established for public benefit and operates in a transparent and accountable manner. The term is used in the context of taxation and financial regulations.

Some possible use cases for this term could include:

1. Non-profit organizations: Entities that are established for public benefit and operate exclusively for charitable, educational, religious, or other similar purposes may be considered "2023 Finance !91 No. 4 of a public character."
2. Government entities: Government agencies or departments that are established for public benefit and operate in a transparent and accountable manner may also fall under this category.
3. Public-private partnerships: Collaborations between public and private entities that are established for public benefit and operate in a transparent and accountable manner may also be considered "2023 F

As we did before, let us use the `ConversationalRetrievalChain` package to give the model context of our previous question so we can add follow up questions.

In [27]:
# use ConversationalRetrievalChain to pass chat history for follow up questions
from langchain.chains import ConversationalRetrievalChain
chat_chain = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever(), return_source_documents=True)

In [28]:
# let's ask the original question "What is llama2?" again
result = chat_chain({"question": question, "chat_history": []})
print(result['answer'])

 The text you provided does not contain the word "ll


In [29]:
# this time we pass chat history along with the follow up so good things should happen
chat_history = [(question, result["answer"])]
followup = "what are its use cases?"
followup_answer = chat_chain({"question": followup, "chat_history": chat_history})
print(followup_answer['answer'])

 I don't know the answer to that question. Llama2 is not a term I am familiar with, and I cannot provide information on its typical use cases.


Further follow ups can be made possible by updating chat_history.

Note that results can get cut off. You may set "max_new_tokens" in the Replicate call above to a larger number (like shown below) to avoid the cut off.

```python
model_kwargs={"temperature": 0.01, "top_p": 1, "max_new_tokens": 1000}
```

In [ ]:
# further follow ups can be made possible by updating chat_history like this:
chat_history.append((followup, followup_answer["answer"]))
more_followup = "what tasks can it assist with?"
more_followup_answer = chat_chain({"question": more_followup, "chat_history": chat_history})
print(more_followup_answer['answer'])

